## Проверяем размеры

In [ ]:
!ls -la ./trainGraph/

In [ ]:
!head ./trainGraph/part-r-00000

In [ ]:
!head ./coreDemography/part-r-00000

In [ ]:
!ls -lh ./trainGraph/part-r-00000

In [ ]:
!ls -lh ./trainGraph/

__27 * 16 = 432 мегабайта - piece of cake!__

### Применительно к нашей задаче

* Первый прогноз можно построить по счетчику общих друзей
* Выданный граф не симетричный


<img src="pics/asymmetrical_graph.jpg" width=700/>

<img src="pics/asymmetrical_graph_4.jpg" width=700/>

## Попытка №1

Рекомендуем в качестве друзей тех, у кого много общих друзей с выбранным пользователем. Для этого втягиваем в память словарь вида [таргет пользователь -> его друзья], и его же в "развернутом виде" [друг ->таргет пользователи, у которых он в друзьях]

### Готовим окружение

In [1]:
import csv, gzip, os, random, numpy, math, timeit, copy, psutil

from collections import defaultdict

# Пути к данным
dataPath = "./"
graphPath = os.path.join(dataPath, "trainGraph")
demographyPath = os.path.join(dataPath, "coreDemography")
csv.field_size_limit(200000)

# Утилита для замера памяти
def memory_usage():
    process = psutil.Process(os.getpid())
    print("Memory usage {0!s} mb.".format(process.memory_info()[0] / 1024 / 1024))

## Попытка №1 - читаем данные

In [ ]:
%%time
mineFriends = defaultdict(list)
friendsOfMine = defaultdict(list)

# Итерируемся по файлам в папке
for file in [f for f in os.listdir(graphPath) if f.endswith("gz")]:
    csv_reader = csv.reader(gzip.open(os.path.join(graphPath, file), "rt"), delimiter='\t')
    # А теперь по строкам в файле
    for line in csv_reader:
        user = int(line[0]) 
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts = friendship.split(",")
            friend = int(parts[0])
            mineFriends[user] += [friend]
            friendsOfMine[friend] += [user]

## Попытка №1 - тревожные сигналы

2 минуты на загрузку 432 мегабайт явно долговато, а что по памяти?

In [ ]:
memory_usage()

Итого два словарика в памяти __уже__ заняли больше трех гигабайт и дальше ворочать их будет неудобно. Но поробуем...

## Попытка №1 - пробуем расчет

In [ ]:
%%time
friendsOfFriends = defaultdict(lambda: defaultdict(int))

for (user, friends) in mineFriends.items():
    for friend in friends:
        for theirFriend in friendsOfMine[friend]:
            friendsOfFriends[user][theirFriend] += 1

## Попытка №1 - работа над ошибками

In [2]:
memory_usage()

Memory usage 62.76171875 mb.


### Применительно к нашей задаче

* Оценим объем данных
* Не забудем о накладных расходах!

## Разбор полетов - исследуем данные

Подумаем о том, как использовать память эффективнее. Для начала давайте попробуем прикинуть базовые показатели графа с которым работаем.

In [3]:
import csv, gzip, os, random, numpy, math, timeit, copy, psutil

from collections import defaultdict

# Пути к данным
dataPath = "./"
graphPath = os.path.join(dataPath, "trainGraph")
demographyPath = os.path.join(dataPath, "CoreDemography")
csv.field_size_limit(200000)

# Утилитка для замера памяти
def memory_usage():
    process = psutil.Process(os.getpid())
    print("Memory usage {0!s} mb.".format(process.memory_info()[0] / 1024 / 1024))

In [4]:
numUsersFrom = 0
numLinks = 0
maxUserIdFrom = 0
maxUserIdTo = 0
allIds = set()

for file in [f for f in os.listdir(graphPath) if f.endswith("gz")]:
    csv_reader = csv.reader(gzip.open(os.path.join(graphPath, file), "rt"), delimiter='\t')
    for line in csv_reader:
        user = int(line[0])
        numUsersFrom += 1
        maxUserIdFrom = max(user, maxUserIdFrom)
        allIds.add(user)
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts = friendship.split(",")
            friend = int(parts[0])
            allIds.add(friend)
            numLinks += 1
            maxUserIdTo = max(maxUserIdTo, friend)
            
print("Num users from {0}, max {1}, num total {2}, max {3}, num links {4}".format(
        numUsersFrom, maxUserIdFrom, len(allIds), maxUserIdTo, numLinks))

Num users from 107474, max 16619036, num total 12417564, max 16619131, num links 36192484


### Оценим объем нужной памяти

На одну связь надо 2 айдишки, которые 4-х байтовый инт

In [5]:
numLinks * (4 + 4) / 1024 / 1024

276.1267395019531

__Почему вместо 276Мб мы получили сразу 3Гб? Почему??!!__

## Разбор полетов - смотрим в код python'а

```c
#define PyDict_MINSIZE 8
typedef struct _dictobject PyDictObject;
struct _dictobject {
  ...
  PyDictEntry *ma_table;
  ...
};

typedef struct {
  Py_ssize_t me_hash;
  PyObject *me_key;
  PyObject *me_value;
} PyDictEntry;
```

## Хэш-таблица с открытой адресацией

<img src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg" width="800" />

## Разбор полетов - считаем накладные расходы

* На запись
  * +8 байт на указатель на запись (PyDictEntry)
  * +4 байта на хэш
  * +8*2 байтов на указатели на ключ/значение
* +30% свободных ячеек
* х2 свободных ячеек при копировании

In [6]:
(numLinks * ((4 + 4) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

1960.4998504638672

Становится яснее!

## А ведь есть еще вопрос локальности данных!


<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Von_Neumann_Architecture.svg/800px-Von_Neumann_Architecture.svg.png" width="600"/>
</center>


<center>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/71982/images/2310246/figure04-caching-pyramid.png" width="600"/>
</center>


| Source   | Latency       |
| ---------|:-------------:|
| L1 Cache |  1-2 ns       |
| L2 Cache |  3-5 ns       |
| L3 Cache |  10-40 ns     |
| DRAM     |  60-100 ns    |



## Боксинг
```c
#define PyObject_HEAD                   \
    _PyObject_HEAD_EXTRA                \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;

typedef struct {
    PyObject_HEAD
    long ob_ival;
} PyIntObject;
```

Кладя инт в "коробку" получаем 20 байт вместо 4-х, итого:

In [7]:
(numLinks * ((20 + 20) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

3065.0068084716795

## Все ли проблемы вскрыли?

__Проблема заполнености__ - общие друзья есть у гораздо большего числа пар пользователей, чем пар пользователей, напрямую связанных дружбой

In [8]:
numUsersFrom * numUsersFrom * 4 / 1024 / 1024

44062.273696899414

In [9]:
360000000 * 360000000 * 4 / 1024 / 1024

494384765625.0

### Применительно к нашей задаче

* Считаем общих друзей только там, где это нужно (входят в тестовое множество)

## Попытка №2

* Выбираем структуру данных
* Втягиваем правильно
* Считаем общих друзей для юзеров из тестового множества
* Профит!

## Выбираем структуры данных

### Словари

* Легко пользоватся.
* Но _ОЧЕНЬ_ много накладных расходов: указатель на запись, указатель на ключ/значение, объектная обертка для ключа и значения.
* Итого: на пару инт-инт _накладных_ расходов 48 байт.
* Данные размазаны по памяти и фрагментированны - кеш процессора резко терят эффективность.

### Списки

* Если ключ инт - можно использовать вместо словаря, как список списков
* НО! Список в питоне это массив указателей, а значит накладные расходы на указатель и объектную обертку ключа. остаются - 20 байт накладных расходов на запись :(.
* Проблема с потерей эффективности кеша сохраняется.

### Массивы numpy

* На пару инт-инт нужно всего 4 байта.
* Кеш процессора работает.
* Но нужно чтобы ключи шли последовательно (к счастью, в наших даных это так почти так: максимальный ид 16619131 при общем числе 12417564).
* Для того чтобы заполнить массив нужно знать его размер изначально.
* Если делать двумерный массив 100к на 16м - будет очень, очень, очень больно...

### Спарс матрицы scipy

* Предназначены для хранения двухмерных матриц с очень большим количеством нулей.
* Эффективные варианты хранят в виде нескольких массивов numpy.
* COO - три массива одинаковой длинны моделируют списов троек [(i,j,v)]. Просто строить, легко интегрировать с пандой.
* CSR - один маленьки массив с индексом начала для каждого ряда (монотонно растет), и два массива равно длинны с ид колонки и значением: [{i : [(j,v)]}]. Более компактна в памяти, быстрые опреации.
* CSC - тоже самое что CSR, но индексированны колонки.

#### CSR matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csr_matrix.png" width="800"/>

#### CSC matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csc_matrix2.png" width="800"/>

## Попытка № 2 - наш выбор

* Инициализруем граф в виде COO матрицы, затем переводим в CSR.
* Развернутый граф получаем с помощью транспонирования.
* Общих друзей считаем умножая матрицы.

#### Умножение матриц? WTF?


<img src="pics/matrix_mult.jpg" width=700/>

## Попытка №2 - готовим окружение

In [10]:
# Почистим не нужное и оставим константы чтоб потом не пресчитывать
del allIds
maxUserId = 16619131
numLinks = 38031656

In [11]:
import numpy
# Работа с матрицами (подсчет общих друзей реализован как умножение матрицы графа 
# самое на себя)
import scipy
from scipy.sparse import coo_matrix, csr_matrix

## Попытка №2 - читаем матрицу

In [12]:
fromUser = numpy.zeros((numLinks), dtype=numpy.int32) 
toUser = numpy.zeros((numLinks), dtype=numpy.int32) 
data = numpy.ones((numLinks), dtype=numpy.int16) 

current = 0

# Итерируемся по файлам в папке... 
for file in [f for f in os.listdir(graphPath) if f.endswith("gz")]:
    csv_reader = csv.reader(gzip.open(os.path.join(graphPath, file), "rt"), delimiter='\t')

    for line in csv_reader:
        user = int(line[0]) 
        maxUserId = max(user, maxUserId)
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts = friendship.split(",")
            # Записываем связь в массивы и двигаем указатель
            fromUser[current] = user
            friend = int(parts[0])
            toUser[current] = friend
            maxUserId = max(friend, maxUserId)
            current += 1
            

Собственно из массивов создаем COO матрицу

In [13]:
fullMatrix = coo_matrix(
    (data, (fromUser, toUser)),
    shape=(maxUserId + 1, maxUserId + 1)).tocsr()

В матрице три массива, соответственно легко померять сколько памяти она занимает

In [14]:
(fullMatrix.data.nbytes + fullMatrix.indptr.nbytes + fullMatrix.indices.nbytes) / (1024 * 1024)

270.49202156066895

In [15]:
(fromUser.nbytes + toUser.nbytes + data.nbytes) / (1024 * 1024)

362.69813537597656

Теперь аккуратно вычистим ненужное

In [16]:
del fromUser
del toUser
del data  

## Попытка №2 - считаем общих друзей для пользователей, по которым необходимо сформировать предсказание

In [17]:
targetUsers = set()
csv_reader = csv.reader(open(os.path.join(dataPath, "prediction.csv")))
for line in csv_reader: 
    targetUsers.add(int(line[0]))

In [18]:
targetUsers

{7503877,
 12713994,
 2523154,
 4915225,
 3375137,
 1638435,
 3178534,
 16613417,
 8683583,
 8552529,
 9109591,
 327774,
 1572963,
 2392166,
 3145831,
 10059881,
 1998960,
 2064509,
 1376382,
 11468926,
 5406848,
 8355970,
 13860997,
 3440774,
 2490506,
 5898383,
 13271188,
 3473565,
 1015967,
 1802401,
 1900708,
 10223781,
 1343657,
 3145897,
 15302833,
 7635129,
 3408060,
 4587711,
 6815937,
 5701835,
 3571916,
 884957,
 10289374,
 2031839,
 1966312,
 3408104,
 4260076,
 2883822,
 1999092,
 7110902,
 10780920,
 9797883,
 4817149,
 14221566,
 5013763,
 9109767,
 5636363,
 11239697,
 688409,
 2162970,
 4784424,
 4817193,
 4489514,
 9535786,
 622893,
 5275959,
 5996855,
 6783289,
 2556220,
 9208129,
 1442118,
 9732422,
 426312,
 10453318,
 6750542,
 14319950,
 9339216,
 229720,
 7471449,
 4981082,
 3637597,
 393576,
 885100,
 1245548,
 5276014,
 5112180,
 7307648,
 2032004,
 885144,
 5570968,
 2097564,
 10224034,
 2556330,
 5505452,
 5898669,
 5243311,
 10420659,
 4325812,
 6259128,
 59

In [19]:
targetUsersMatrix = copy.deepcopy(fullMatrix)

for i in range(maxUserId + 1):
    if i not in targetUsers:
        ptr = targetUsersMatrix.indptr[i]
        ptr_next = targetUsersMatrix.indptr[i+1]
        if ptr != ptr_next:
            targetUsersMatrix.data[ptr:ptr_next].fill(0)

In [20]:
targetUsersMatrix.eliminate_zeros()

In [21]:
(targetUsersMatrix.data.nbytes + targetUsersMatrix.indptr.nbytes + targetUsersMatrix.indices.nbytes) / (1024 * 1024)

79.27946090698242

In [22]:
%%time
commonFriends = targetUsersMatrix.dot(fullMatrix)

CPU times: user 22.5 s, sys: 840 ms, total: 23.4 s
Wall time: 23.4 s


In [23]:
type(commonFriends)

scipy.sparse.csr.csr_matrix

In [24]:
memory_usage()

Memory usage 1583.078125 mb.


In [25]:
len(targetUsers)

7088

## Что будет, если мы попробуем посчитать общих друзей для всех пользователей

In [26]:
%%time
commonFriends = fullMatrix.dot(fullMatrix)

CPU times: user 5min 16s, sys: 14.1 s, total: 5min 30s
Wall time: 5min 30s


In [ ]:
memory_usage()



## Семинар

Для targetUsers посчитать оценку  возраста, как средние от возрастов друзей

In [ ]:
demographyPath

In [ ]:
birthDates = numpy.zeros(maxUserId + 1, dtype=numpy.int32)

# Iterate all files in demography path
for file in [f for f in os.listdir(demographyPath) if f.endswith("gz")]:
    csvinput = gzip.open(os.path.join(demographyPath, file), "rt")
    csv_reader = csv.reader(csvinput, delimiter='\t')
   
    # Extract birth date from each line
    for line in csv_reader:
        user = int(line[0])
        birthDates[user] = int(line[2]) if line[2] != '' else 0

In [ ]:
print ("Memory used by demography: %s (mb)" % (birthDates.nbytes / (1024 * 1024)))